In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Data Cleaning

In [116]:
excel = pd.ExcelFile('HainanClean_New.xlsx')
hainan = excel.parse("fulldf")
hainan
hainan.columns = hainan.columns.str.replace('  ', '_')
hainan.columns = hainan.columns.str.replace(' ', '_')
hainan.columns = hainan.columns.str.replace('(', '')
hainan.columns = hainan.columns.str.replace('（', '')
hainan.columns = hainan.columns.str.replace(')', '')

d = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
     'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
hainan.Month = hainan.Month.map(d)
hainan.BioCNG_Produced_Nm3 = hainan.BioCNG_Produced_m3.shift(-15)
hainan.drop(hainan.tail(15).index,inplace=True)
hainan = hainan[np.isfinite(hainan['Month'])]
hainan['BioCNG_cumsum'] = hainan.BioCNG_Produced_m3.cumsum()
hainan.columns
hainan = hainan[np.isfinite(hainan['Lemon_waste_t'])]
hainan = hainan[np.isfinite(hainan['Percolate_t'])]
hainan = hainan.replace(' ',0)
hainan = hainan.replace('',0)
hainan = hainan.replace('  ',0)
hainan.columns

Index(['Year', 'Month', 'Day', 'Month_#', 'Day_#', 'Raw_Biogas_Produced_m3',
       'BioCNG_Produced_m3', 'BioCNG_Sold_m3', 'Vehicle_use_m3',
       'Liquid_Fertilizer_Produced_t', 'Solid_fertilizer_produced_t',
       'Wastewater_flow_to_WWTP_unit?', 'Solid_residues_kg', 'Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t', '50%_NaOH/kg', 'FeCl2/kg', 'PAM/kg',
       'Defoamer/kg', 'Project_electricity_use/kWh',
       'Office_space_electricity_use/kWh', 'Water/m3', 'Diesel/L',
       'BioCNG_cumsum'],
      dtype='object')

# Preliminary Work

In [117]:
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
print('train data len:',len(hainan_train))
print('test data len:',len(hainan_test))

train data len: 1098
test data len: 275


In [ ]:
data = randn(100,5)
y = data[:,0]
x = data[:,1:]

mymodel = ols.ols(y,x,'BioCNG_cumsum',['Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t'])
mymodel.p               # return coefficient p-values
mymodel.summary()       # print results

In [118]:
#ols
hainan_ols = ols("BioCNG_cumsum ~ Pig_Manure_t + Cassava_t + Fish_waste_water_t +\
                    Kitchen_food_waste_t + Municipal_fecal_residue_t + Tea_waste_t +\
                    Chicken_litter_t + Bagasse_feed_t + Alcohol_waste_t +\
                    Chinese_medicine_waste_t + Energy_grass_t + Banana_fruit_shafts_t +\
                    Lemon_waste_t + Percolate_t + Other_waste_t", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          BioCNG_cumsum   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     379.5
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:39:52   Log-Likelihood:                -16461.
No. Observations:                1098   AIC:                         3.295e+04
Df Residuals:                    1082   BIC:                         3.303e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                  1.332e+06   4.99e+04     26.689      0.000    1.23e+06    1.43e+06
Pig_Manure_t               3.439e+04   2679.829     12.831      0.000    2.91e+04    3.96e+04
Cassava_t                   -37.8089   1028.101     -0.037      0.971   -2055.106    1979.488
Fish_waste_water_t         2592.6077   4675.111      0.555      0.579   -6580.703    1.18e+04
Kitchen_food_waste_t        3.03e+04   2528.851     11.982      0.000    2.53e+04    3.53e+04
Municipal_fecal_residue_t  6649.6614   1040.901      6.388      0.000    4607.248    8692.075
Tea_waste_t                2.742e+04   8527.706      3.216      0.001    1.07e+04    4.42e+04
Chicken_litter_t             1.5e+04   4132.160      3.629      0.000    6888.756    2.31e+04
Bagasse_feed_t            -1.396e+04   1686.100     -8.282      0.000   -1.73e+04   -1.07e+04
Alcohol_waste_t            3.477e+04   6745.023      5.154      0.000    2.15e+04     4.8e+04
Chinese_medicine_waste_t   3.023e+04   2.24e+04      1.352      0.177   -1.37e+04    7.41e+04
Energy_grass_t             3.115e+04   1.71e+04      1.827      0.068   -2309.808    6.46e+04
Banana_fruit_shafts_t     -1637.5728   6926.030     -0.236      0.813   -1.52e+04     1.2e+04
Lemon_waste_t              -1.01e+04   2.19e+04     -0.460      0.645   -5.31e+04    3.29e+04
Percolate_t                2248.8491    289.247      7.775      0.000    1681.301    2816.397
Other_waste_t              6060.7894   2577.861      2.351      0.019    1002.616    1.11e+04
==============================================================================
Omnibus:                       25.688   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.304
Skew:                           0.267   Prob(JB):                     2.37e-05
Kurtosis:                       2.575   Cond. No.                         348.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [119]:
#ols
hainan_ols = ols("Liquid_Fertilizer_Produced_t ~ Pig_Manure_t + Cassava_t + Fish_waste_water_t +\
                    Kitchen_food_waste_t + Municipal_fecal_residue_t + Tea_waste_t +\
                    Chicken_litter_t + Bagasse_feed_t + Alcohol_waste_t +\
                    Chinese_medicine_waste_t + Energy_grass_t + Banana_fruit_shafts_t +\
                    Lemon_waste_t + Percolate_t + Other_waste_t", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     Liquid_Fertilizer_Produced_t   R-squared:                       0.089
Model:                                      OLS   Adj. R-squared:                  0.077
Method:                           Least Squares   F-statistic:                     7.067
Date:                          Fri, 09 Mar 2018   Prob (F-statistic):           6.14e-15
Time:                                  16:39:52   Log-Likelihood:                -5876.9
No. Observations:                          1095   AIC:                         1.179e+04
Df Residuals:                              1079   BIC:                         1.187e+04
Df Model:                                    15                                         
Covariance Type:                      nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    21.1928      3.299      6.424      0.000      14.720      27.666
Pig_Manure_t                  0.8386      0.177      4.729      0.000       0.491       1.187
Cassava_t                     0.3790      0.068      5.580      0.000       0.246       0.512
Fish_waste_water_t            0.8060      0.309      2.610      0.009       0.200       1.412
Kitchen_food_waste_t         -0.2495      0.167     -1.492      0.136      -0.578       0.079
Municipal_fecal_residue_t     0.1371      0.069      1.993      0.047       0.002       0.272
Tea_waste_t                   0.3706      0.564      0.657      0.511      -0.736       1.477
Chicken_litter_t              0.4978      0.273      1.822      0.069      -0.038       1.034
Bagasse_feed_t                0.2003      0.111      1.797      0.073      -0.018       0.419
Alcohol_waste_t              -0.7731      0.446     -1.735      0.083      -1.647       0.101
Chinese_medicine_waste_t     -4.2242      1.511     -2.796      0.005      -7.189      -1.259
Energy_grass_t               -1.2868      1.127     -1.142      0.254      -3.497       0.924
Banana_fruit_shafts_t        -0.7202      0.458     -1.572      0.116      -1.619       0.179
Lemon_waste_t                 0.1052      1.450      0.073      0.942      -2.740       2.950
Percolate_t                  -0.0205      0.019     -1.067      0.286      -0.058       0.017
Other_waste_t                 0.2830      0.170      1.662      0.097      -0.051       0.617
==============================================================================
Omnibus:                      263.328   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              527.236
Skew:                           1.390   Prob(JB):                    3.25e-115
Kurtosis:                       4.955   Cond. No.                         346.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [120]:
#ols
hainan_ols = ols("Solid_fertilizer_produced_t ~ Pig_Manure_t + Cassava_t + Fish_waste_water_t +\
                    Kitchen_food_waste_t + Municipal_fecal_residue_t + Tea_waste_t +\
                    Chicken_litter_t + Bagasse_feed_t + Alcohol_waste_t +\
                    Chinese_medicine_waste_t + Energy_grass_t + Banana_fruit_shafts_t +\
                    Lemon_waste_t + Percolate_t + Other_waste_t", data=hainan_train).fit()
hainan_ols_summary = hainan_ols.summary()
hainan_ols_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Solid_fertilizer_produced_t   R-squared:                       0.153
Model:                                     OLS   Adj. R-squared:                  0.141
Method:                          Least Squares   F-statistic:                     13.01
Date:                         Fri, 09 Mar 2018   Prob (F-statistic):           1.90e-30
Time:                                 16:39:53   Log-Likelihood:                -5063.2
No. Observations:                         1098   AIC:                         1.016e+04
Df Residuals:                             1082   BIC:                         1.024e+04
Df Model:                                   15                                         
Covariance Type:                     nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     2.3598      1.549      1.524      0.128      -0.679       5.399
Pig_Manure_t                 -0.2317      0.083     -2.787      0.005      -0.395      -0.069
Cassava_t                     0.0503      0.032      1.577      0.115      -0.012       0.113
Fish_waste_water_t            0.3523      0.145      2.429      0.015       0.068       0.637
Kitchen_food_waste_t          0.0540      0.078      0.689      0.491      -0.100       0.208
Municipal_fecal_residue_t     0.0617      0.032      1.911      0.056      -0.002       0.125
Tea_waste_t                  -0.5318      0.265     -2.010      0.045      -1.051      -0.013
Chicken_litter_t              0.1468      0.128      1.145      0.252      -0.105       0.398
Bagasse_feed_t                0.1910      0.052      3.651      0.000       0.088       0.294
Alcohol_waste_t               0.0008      0.209      0.004      0.997      -0.410       0.411
Chinese_medicine_waste_t      0.4452      0.694      0.641      0.521      -0.917       1.807
Energy_grass_t                0.1372      0.529      0.259      0.795      -0.901       1.175
Banana_fruit_shafts_t        -0.0398      0.215     -0.185      0.853      -0.461       0.382
Lemon_waste_t                 0.4680      0.680      0.688      0.492      -0.867       1.803
Percolate_t                   0.0504      0.009      5.612      0.000       0.033       0.068
Other_waste_t                 0.1334      0.080      1.668      0.096      -0.024       0.290
==============================================================================
Omnibus:                     1056.233   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            53008.866
Skew:                           4.414   Prob(JB):                         0.00
Kurtosis:                      35.874   Cond. No.                         348.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

From the ols results based on three different dependent variable, we can see that only the first one (cumulative sum of BioCNG) has a relatively high value for R-squared. So We decided to focus on the first one.

In [121]:
# Predict on the training data
X_train = hainan_train[['Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t']]
y_train = hainan_train.BioCNG_cumsum
y_pred_train = hainan_ols.predict(X_train)

# Compute the root-mean-square of training data
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('Training error:',rms_train/sum(y_train))

# Predict on the test data
X_test = hainan_test[['Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t']]
y_test = hainan_test.BioCNG_cumsum
y_pred_test = hainan_ols.predict(X_test)

# Compute the root-mean-square of test data
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print('Testing error:',rms_test/sum(y_test))

hainan_train

Training error: 0.00114203353376
Testing error: 0.00453542212463


,Year,Month,Day,Month_#,Day_#,Raw_Biogas_Produced_m3,BioCNG_Produced_m3,BioCNG_Sold_m3,Vehicle_use_m3,Liquid_Fertilizer_Produced_t,...,Other_waste_t,50%_NaOH/kg,FeCl2/kg,PAM/kg,Defoamer/kg,Project_electricity_use/kWh,Office_space_electricity_use/kWh,Water/m3,Diesel/L,BioCNG_cumsum
660,2016,2.0,20,22,661,7386.0,3960.00,0.00,NaN,112.60,...,0.00,0.00,0,0.0,0,4995,98.0,11,200.00,2121301.00
564,2015,11.0,16,19,565,2478.0,1482.00,4571.00,NaN,0.00,...,0.00,0.00,0,0.0,0,3361,137.0,7,0.00,1694999.00
898,2016,10.0,15,30,899,2292.0,1337.00,0.00,NaN,0.00,...,0.00,0.00,0,0.0,0,3578,131.0,56,0.00,3009981.00
1008,2017,2.0,2,34,1009,10745.0,6156.00,4150.00,NaN,0.00,...,0.00,0.00,0,0.0,0,8529,30.0,15,0.00,3597681.00
79,2014,7.0,19,3,80,0.0,0.00,0.00,NaN,0.00,...,0.00,0.00,0,0.0,0,0,0.0,0,0.00,178166.00
1344,2018,1.0,4,45,1345,16067.0,10270.00,9058.00,603.0,0.00,...,0.00,75.00,0,0.0,0,13369,78.0,155,133.00,6980766.47
1260,2017,10.0,12,42,1261,17730.0,11333.00,13191.00,463.0,0.00,...,0.00,75.00,0,0.0,0,15055,150.0,115,359.92,6082287.78
366,2015,5.0,2,13,367,7032.0,3660.00,3444.00,NaN,36.76,...,0.00,0.00,0,0.0,0,5889,0.0,24,0.00,1070930.00
840,2016,8.0,18,28,841,0.0,0.00,0.00,NaN,0.00,...,0.00,0.00,0,0.0,0,0,0.0,0,0.00,2875098.00
1127,2017,6.0,1,38,1128,18744.0,11476.00,7702.00,NaN,77.82,...,0.00,100.00,0,0.0,0,14565,224.0,86,0.00,4590919.00


# Training Algorithms

1. Logistic regression
2. SVM
3. Perceptron
4. kNN
5. Random Forest
6. xgBoost

In [122]:
#logistic regression
hainan['BioCNG_cumsum'] = pd.cut(hainan['BioCNG_cumsum'], bins=3, labels=[0, 1, 2])
hainan_train, hainan_test = train_test_split(hainan, test_size=0.2)
y_train = hainan_train.BioCNG_cumsum
y_test = hainan_test.BioCNG_cumsum
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(logreg.score(X_train, y_train)))
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of logistic regression classifier on train set: 0.53
Accuracy of logistic regression classifier on test set: 0.51
[[140   0   0]
 [ 87   0   0]
 [ 48   0   0]]


In [123]:
# SVM
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)  
y_pred_test = clf.predict(X_test)
print('Accuracy of SVM on train set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of SVM on test set: {:.2f}'.format(clf.score(X_test, y_test)))

Accuracy of SVM on train set: 0.85
Accuracy of SVM on test set: 0.48


In [124]:
# perceptron
from sklearn.linear_model import perceptron
net = perceptron.Perceptron(n_iter=100, verbose=0, random_state=None, fit_intercept=True, eta0=0.002)
net.fit(X_train, y_train)
y_pred_test = net.predict(X_test)
print('Accuracy of perceptron on train set: {:.2f}'.format(net.score(X_train, y_train)))
print('Accuracy of perceptron on test set: {:.2f}'.format(net.score(X_test, y_test)))

Accuracy of perceptron on train set: 0.30
Accuracy of perceptron on test set: 0.33


/Users/anna/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [125]:
# kNN
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(hainan[['BioCNG_cumsum', 'Pig_Manure_t',
       'Cassava_t', 'Fish_waste_water_t', 'Kitchen_food_waste_t',
       'Municipal_fecal_residue_t', 'Tea_waste_t', 'Chicken_litter_t',
       'Bagasse_feed_t', 'Alcohol_waste_t', 'Chinese_medicine_waste_t',
       'Energy_grass_t', 'Banana_fruit_shafts_t', 'Lemon_waste_t',
       'Percolate_t', 'Other_waste_t']])
hainan_normalized = pd.DataFrame(np_scaled)
hainan_normalized_train, hainan_normalized_test = train_test_split(hainan_normalized, test_size=0.2)

Xnor_train = hainan_normalized_train.iloc[:, 1:16]
ynor_train = hainan_normalized_train[0]
Xnor_test = hainan_normalized_test.iloc[:, 1:16]
ynor_test = hainan_normalized_test[0]
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(Xnor_train, ynor_train)
y_pred_test = knn.predict(Xnor_test)
print('Accuracy of kNN on train set: {:.2f}'.format(knn.score(Xnor_train, ynor_train)))
print('Accuracy of kNN on test set: {:.2f}'.format(knn.score(Xnor_test, ynor_test)))

Accuracy of kNN on train set: 0.89
Accuracy of kNN on test set: 0.80


In [126]:
# xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred_test = xgb.predict(X_test)
print('Accuracy of XGboost on train set: {:.2f}'.format(xgb.score(X_train, y_train)))
print('Accuracy of XGboost on test set: {:.2f}'.format(xgb.score(X_test, y_test)))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
print(confusion_matrix)

Accuracy of XGboost on train set: 0.63
Accuracy of XGboost on test set: 0.49
[[127  13   0]
 [ 77   8   2]
 [ 40   7   1]]


In [127]:
# random forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000)
random_forest.fit(X_train, y_train)
print('Accuracy of Random Forest on train set: {:.2f}'.format(random_forest.score(X_train, y_train)))
print('Accuracy of Random Forest on test set: {:.2f}'.format(random_forest.score(X_test, y_test)))

Accuracy of Random Forest on train set: 0.97
Accuracy of Random Forest on test set: 0.48


From the accuracy results of the above algorithms training, we can see that except for kNN, all of them have relatively good results for accuracy around 0.5.